<a href="https://colab.research.google.com/github/Sakshipingale13/Sentiment-Analysis/blob/main/Sentimental_Analysis_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install kaggle


Upload your kaggle dataset

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Import Twitter dataset

In [3]:
# API TO FETCH THE DATASET FROM KAGGLE
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 99% 80.0M/80.9M [00:01<00:00, 70.5MB/s]
100% 80.9M/80.9M [00:01<00:00, 71.8MB/s]


In [4]:
from zipfile import ZipFile  # Correct import statement

dataset = "/content/sentiment140.zip"

with ZipFile(dataset, 'r') as zip:
    zip.extractall()
    print("Done")


Done


Import the libraries

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
# printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Preprocessing

In [8]:
# load the dataset from csv file to pands dataframe
twitter_data=pd.read_csv("/content/training.1600000.processed.noemoticon.csv" ,encoding='ISO-8859-1')

In [9]:
# checking the number of columns and rows
twitter_data.shape

(1599999, 6)

In [10]:
# printing the first 5 rows
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [11]:
# renaming the column names and readinf=g it again

column_name=['target','id', 'date','flag','user','text']
twitter_data=pd.read_csv("/content/training.1600000.processed.noemoticon.csv" , names=column_name,encoding='ISO-8859-1')

In [12]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [13]:
twitter_data.shape

(1600000, 6)

In [14]:
# checkimg the missing values in dataframe
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [15]:
# checking the distribution of target column
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

converting  4= 1

In [16]:
twitter_data['target'].replace({4:1},inplace=True)

In [17]:
# checking the distribution of target column
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

**Stemming**

In [18]:
# Initialize the PorterStemmer
port_stem = PorterStemmer()

def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [20]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [21]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [22]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [23]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [24]:
# separating the data and label
X= twitter_data['stemmed_content'].values
Y= twitter_data['target'].values

In [25]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [26]:
print(Y)

[0 0 0 ... 1 1 1]


In [28]:
# split  the data into training and testing set
X_train ,X_test, Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=3)

In [29]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [30]:
print(X_test)

['hate dog bark someth noth happen scari time tri sleep'
 'willi day either support cuz like guy' 'jordalynn yah pretti mess' ...
 'use spare usb flash drive virtual ram pc got gb memori'
 'sistatreen jealou miss beach'
 'sakura kurosaki prob hope good one lt ooh kind cake love cake']


In [31]:
print(X_train)

['arm fell comput chair go lean' 'chrishillfan got media next lucki'
 'suck havin watch yur mother sick cnt help' ...
 'melosm oooop sorri non exist miss call'
 'frowzl accord j rg kachelmann start rain tomorrow clock morn let hope right ja komm auch mit'
 'alcarlton lol idea mate go buy cloth much simpler option']


In [32]:
# converting textual data into numerical data
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [33]:
print(X_test)

  (0, 415689)	0.2518922757443892
  (0, 408499)	0.21003368771385605
  (0, 375101)	0.28488347461012686
  (0, 371029)	0.2464047959343191
  (0, 354711)	0.38027426988122187
  (0, 294474)	0.2984550266257167
  (0, 160330)	0.2625869593536074
  (0, 159203)	0.30050250593214706
  (0, 105377)	0.3245022701383146
  (0, 33329)	0.5023175290684292
  (1, 442056)	0.5541089394462928
  (1, 389414)	0.42351953294380973
  (1, 234202)	0.23372878235800623
  (1, 156037)	0.3169093785810737
  (1, 115722)	0.392787042356637
  (1, 93683)	0.21941299197386777
  (1, 87100)	0.39510766737101444
  (2, 452062)	0.4488951980065795
  (2, 322579)	0.30380202466576606
  (2, 261613)	0.3834893477036243
  (2, 201868)	0.7477521990696274
  (3, 456101)	0.9045922983924057
  (3, 267239)	0.42627781280421434
  (4, 435673)	0.17094573599581595
  (4, 368276)	0.24078151125805283
  :	:
  (319997, 432040)	0.3449292201083686
  (319997, 426743)	0.206243445111222
  (319997, 426714)	0.35562094758353474
  (319997, 377504)	0.34307286039499907
  (31999

In [34]:
print(X_train)

  (0, 230023)	0.5302789468855067
  (0, 149604)	0.18999894424903027
  (0, 66262)	0.4703657150050708
  (0, 79295)	0.3591444105830402
  (0, 130345)	0.39805809775478695
  (0, 23107)	0.4171639689388593
  (1, 243427)	0.35968353780563106
  (1, 288268)	0.28409690701894913
  (1, 258288)	0.42799264451062413
  (1, 151539)	0.22265959091242618
  (1, 71762)	0.7464326512642685
  (2, 163634)	0.2761690912767651
  (2, 76994)	0.437205368032008
  (2, 366626)	0.2737474570064778
  (2, 274638)	0.3245662122941835
  (2, 457071)	0.49837923199930634
  (2, 436235)	0.22572360152384552
  (2, 160639)	0.4212655595518347
  (2, 386886)	0.2747790530029926
  (3, 172089)	0.26317971923417777
  (3, 384525)	0.33882627077985117
  (3, 256931)	0.4050476960176577
  (3, 438627)	0.26510430687972497
  (3, 365775)	0.3124620940507158
  (3, 412683)	0.3032716769864438
  :	:
  (1279998, 139300)	0.3779561489329452
  (1279998, 220549)	0.3779561489329452
  (1279998, 269095)	0.29047254123711924
  (1279998, 338984)	0.2926984744274037
  (1279

**Training the model**

Logistic Model

In [35]:
model = LogisticRegression(max_iter=1000)

In [36]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

**Model Evaluation**

In [37]:
# Accuracy on trainnig data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train,X_train_prediction)

In [38]:
print("The accuracy of training data is::" ,training_data_accuracy)

The accuracy of training data is:: 0.81000390625


In [39]:
# Accuracy on trainnig data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test,X_test_prediction)

In [40]:
print("The accuracy of training data is::" ,test_data_accuracy)

The accuracy of training data is:: 0.7785375


Saving the trained model

In [41]:
import pickle

In [42]:
filename='trained_model.sav'
pickle.dump(model,open(filename,'wb'))

using the saved model for fututre predictions

In [43]:
# loading the saved model
loaded_model=pickle.load(open('/content/trained_model.sav','rb'))

In [44]:
X_new = X_test[200]
print(Y_test[200])

prediction=model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print("Negative Tweet")

else:
  print("Positive Tweet")

0
[0]
Negative Tweet


In [47]:
X_new = X_test[100]
print(Y_test[100])

prediction=model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print("Negative Tweet")

else:
  print("Positive Tweet")

1
[1]
Positive Tweet
